In [236]:
import os
import pickle
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import pipeline

In [238]:
# Load web data
urls = [
    "https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
]
loader = WebBaseLoader(urls)
data = loader.load()

In [240]:
# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(data)

In [242]:
# Create embeddings
embeddings = HuggingFaceEmbeddings()
vector_index = FAISS.from_documents(docs, embeddings)


C:\Users\91750\AppData\Local\Temp\ipykernel_22364\3695376538.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


In [244]:
# Store the vector index locally
file_path = "vector_index.pkl"
with open(file_path, "wb") as f:
    pickle.dump(vector_index, f)


In [246]:
# Load stored vector index
if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        vector_index = pickle.load(f)

In [248]:
# Initialize Hugging Face QA model
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

Device set to use cpu


In [250]:
# Query the model
query = "What is the price of Tiago iCNG?"
retrieved_docs = vector_index.similarity_search(query, k=1)
context = retrieved_docs[0].page_content if retrieved_docs else ""

In [252]:
# Get answer from the QA model
result = qa_pipeline(question=query, context=context)
print("Answer:", result["answer"])

Answer: between Rs 6.55 lakh and Rs 8.1 lakh
